In [1]:
# imports
import networkx as nx
import numpy as np
from itertools import combinations
from collections import deque
import cluster_refinement as refine
import spectral_embedding as se
import pathfinders
import random
import example_graphs as examples
import visual_aids as vis
import random
# other
strategies = [
        {'name': 'boundary first', 'weights': {'boundary': 1.0, 'deg_diff': 0.0}},
        {'name': 'boundary last', 'weights': {'boundary': -1.0, 'deg_diff': 0.0}},
        {'name': 'deg diff descending', 'weights': {'boundary': 0.0, 'deg_diff': 1.0}},
        {'name': 'deg diff ascending', 'weights': {'boundary': 0.0, 'deg_diff': -1.0}},
        {'name': '(1.0, 1.0)', 'weights': {'boundary': 1.0, 'deg_diff': 1.0}},
        {'name': '(-1.0, -1.0)', 'weights': {'boundary': -1.0, 'deg_diff': -1.0}},
        {'name': 'random', 'weights': None},
        {'name': 'fiedler', 'weights': None},
]

In [2]:
# test data : ham_tests (list), nonham_tests (list of (name, graph))
def generate_test_data(n):
    from data import test_data
    from example_graphs import generate_classic_nonhamiltonian_graphs
    ham_tests = list(test_data.values())
    nonham_tests = list(generate_classic_nonhamiltonian_graphs(n).items())
    return ham_tests, nonham_tests
ham_tests, nonham_tests = generate_test_data(20)
def generate_random_sparse_graphs(N, n, p):
    data = []
    from example_graphs import test_graph_sparsity
    for _ in range(N):
        data.append(test_graph_sparsity(n,p))
    return data

In [3]:
def recursive_cheeger_split(G, frontier=None, min_size=2, cond_threshold=1, symmetry_tol=1, max_depth=3, current_depth=0, init_n=None):
    if frontier is None:
        frontier = []
    n = G.number_of_nodes()
    if init_n is None:
        init_n = n
    if n == 0 or n <= min_size or n < init_n / 4 or current_depth >= max_depth:
        return {
            'nodes': set(G.nodes()),
            'frontier': frontier,
            'graph': G,
            'spectral': None,
            'cheeger': None,
            'symmetric': True,
            'children': None
        }
    
    cond, cut_set = refine.cheeger_cut(G)
    boundary = refine.boundary_edges(G, cut_set)
    new_frontier = frontier.copy() 
    new_frontier.append(boundary)
    
    if cond is None or cond > cond_threshold or cut_set == set(G.nodes()) or not cut_set:
        return {
            'nodes': set(G.nodes()),
            'frontier': new_frontier,
            'graph': G,
            'spectral': None,
            'cheeger': None,
            'symmetric': True,
            'children': None
        }
    
    comp_left = G.subgraph(cut_set).copy()
    comp_right = G.subgraph(set(G.nodes()) - cut_set).copy()
    
    left_spec = se.spectral_radius(comp_left)
    right_spec = se.spectral_radius(comp_right)
    left_cheeger, _ = refine.cheeger_cut(comp_left)
    right_cheeger, _ = refine.cheeger_cut(comp_right)
    
    spectral_tuple = (left_spec, right_spec)
    cheeger_tuple = (left_cheeger, right_cheeger)
    
    spec_diff = abs(left_spec - right_spec) if left_spec is not None and right_spec is not None else float('inf')
    cheeger_diff = abs(left_cheeger - right_cheeger) if left_cheeger is not None and right_cheeger is not None else float('inf')
    is_symmetric = (spec_diff < symmetry_tol) and (cheeger_diff < symmetry_tol)
    
    left_subtree = recursive_cheeger_split(comp_left, new_frontier, min_size, cond_threshold, symmetry_tol, max_depth, current_depth + 1, init_n)
    right_subtree = recursive_cheeger_split(comp_right, new_frontier, min_size, cond_threshold, symmetry_tol, max_depth, current_depth + 1, init_n)
    
    if is_symmetric:
        return {
            'nodes': set(G.nodes()),
            'frontier': new_frontier,
            'graph': G,
            'spectral': spectral_tuple,
            'cheeger': cheeger_tuple,
            'symmetric': True,
            'children': [left_subtree, right_subtree]
        }
    
    return {
        'nodes': set(G.nodes()),
        'frontier': new_frontier,
        'graph': G,
        'spectral': spectral_tuple,
        'cheeger': cheeger_tuple,
        'symmetric': False,
        'children': [left_subtree, right_subtree]
    }

In [ ]:
for name, graph in nonham_tests:
    closed = refine.bondy_chvatal_closure(graph)
    reduced = refine.aggressive_pruning_ordered(closed, weights={'boundary': -1.0, 'deg_diff': 0.0})
    

In [4]:
G = random.choice(ham_tests)
data = se.fast_eigen_decomp(G)
labels = se.get_partitions(data)

graph size 3000 > 750: using k = 81 with QR approximation
found clustering behavior on signless laplacian eigenvalues for 44 clusters


In [ ]:
clusters = refine.labels_to_clusters(G, labels)
closed_clusters = []
for cluster in clusters.values():
    cste, cut = refine.cheeger_cut(cluster)
    if cste < 0.5:
        
    else:
        cl_cluster = refine.bondy_chvatal_closure(cluster)
        reduced = refine.aggressive_pruning_ordered(cl_cluster, weights={'boundary': -1.0, 'deg_diff': 0.0})

65
82
56
71
96
65
72
72
84
51
64
65
71
60
84
52
84
59
59
84
71
65
76
65
64
67
63
51
51
69
73
69
57
73
86
61
80
63
79
55
68
64
81
53


In [5]:
def contract_cycles(G, cycles):
    # Nodes that are in a cycle get mapped to a supernode label.
    mapping = {}
    for i, cycle in enumerate(cycles):
        # Create a label for the supernode representing the cycle.
        supernode = f"supernode_{i}"
        for node in cycle:
            mapping[node] = supernode
    # For nodes not included in any cycle, map them to themselves.
    for node in G.nodes():
        if node not in mapping:
            mapping[node] = node

    # Now build the new graph using the mapping.
    H = nx.Graph()
    for u, v, data in G.edges(data=True):
        # Determine new endpoints based on the mapping.
        new_u = mapping[u]
        new_v = mapping[v]
        # Avoid self-loops (edges within the same supernode)
        if new_u == new_v:
            continue
        # Add the edge to the new graph. If multiple edges emerge between the same nodes,
        # you might want to aggregate or simply ignore duplicates.
        if H.has_edge(new_u, new_v):
            # Optionally, aggregate attributes or count multiplicity.
            continue
        H.add_edge(new_u, new_v, **data)
    return H

Gcontracted = contract_cycles(G, cycles)

In [ ]:
import networkx as nx

def has_hamiltonian_path_via_cycle(G, A, cycle_nodes, B):
    """
    Check whether there is a Hamiltonian path from A to B through the cycle.
    
    The idea is to verify that the cycle (or the set of nodes from which
    the supernode was built) can be traversed in such a way that A and B
    (or their connection points) are linked by a path that visits every vertex 
    of the cycle once.
    
    For demonstration purposes, this function returns True if:
      - A and B both connect to some nodes in cycle_nodes
      - And (optionally) if the connections appear 'symmetric enough'
      
    In a rigorous implementation, you would apply a Pósa rotation technique
    or another method to verify the Hamiltonicity condition.
    """
    # For simplicity, check that at least one node in the cycle connects to A
    # and one (potentially different) node connects to B.
    connects_A = any(G.has_edge(A, node) for node in cycle_nodes)
    connects_B = any(G.has_edge(B, node) for node in cycle_nodes)
    
    # This is a very lenient condition. Replace with Pósa's rotation logic as needed.
    return connects_A and connects_B

def contract_supernodes_via_posa(G, cycles):
    """
    For each cycle (a list of nodes) provided in `cycles`, we assume it has been
    contracted to a supernode in G with the name 'supernode_{i}'.
    
    This function goes over each such cycle/supernode and attempts to contract
    the chain A - supernode - B into a single edge A-B. The contraction is done
    only when a Hamiltonian path exists via the cycle as confirmed by a Pósa rotation
    based check (or a simplified version thereof).
    
    The function returns a new graph with those valid contractions.
    """
    # Create a mapping from the cycle list to its supernode label.
    mapping = {}
    for i, cycle in enumerate(cycles):
        supernode = f"supernode_{i}"
        for node in cycle:
            mapping[node] = supernode
    
    H = nx.Graph()
    # First, build graph H by mapping nodes from G to supernodes when needed.
    for u, v, data in G.edges(data=True):
        new_u = mapping.get(u, u)
        new_v = mapping.get(v, v)
        if new_u != new_v:
            H.add_edge(new_u, new_v, **data)
    
    # Now look for cases where a supernode connects exactly two distinct nodes: A and B.
    to_contract = []
    for node in list(H.nodes()):
        # Only check nodes that are supernodes.
        if isinstance(node, str) and node.startswith("supernode_"):
            neighbors = list(H.neighbors(node))
            if len(neighbors) == 2:
                A, B = neighbors
                # Retrieve the original cycle that formed this supernode.
                cycle_index = int(node.split("_")[1])
                cycle_nodes = cycles[cycle_index]
                # Use Pósa's rotation (or a simplified check) to verify Hamiltonian connection.
                if has_hamiltonian_path_via_cycle(G, A, cycle_nodes, B):
                    to_contract.append((node, A, B))
    
    # For each valid contraction, remove the supernode and add a direct edge A-B.
    for supernode, A, B in to_contract:
        if H.has_node(supernode):
            H.remove_node(supernode)
            if not H.has_edge(A, B):
                H.add_edge(A, B)
    
    return H


In [6]:
def loop_extract(G, contracted, n):
    A = se.to_adjacency(contracted)
    _,fiedler,_ = se.power_method(A)
    cycles = pathfinders.greedy_partial_cycle_extractor(G, fiedler)
    contracted_new = contract_cycles(G, cycles)
    if len(contracted)<5 or np.abs(contracted_new.number_of_nodes()-contracted.number_of_nodes())<n:
        return contracted
    return loop_extract(G, contracted_new, n)
n = np.log(G.number_of_nodes())*200
contracted = loop_extract(G, G, n)

iteration 1
extracted cycle: [2000, 1792, 1791, 1790, 1789, 1203, 1956, 1957, 1236, 15, 1980, 1981, 1242, 1982, 1243, 1983, 1244, 1984, 503, 838, 1989, 1245, 1990, 1991, 1992, 1246, 1993, 1994, 1995, 1171, 1691, 1692, 1693, 1173, 1917, 1918, 499, 1919, 500, 1920, 1921, 1229, 1922, 501, 1988, 1184, 1725, 773, 774, 1941, 998, 1202, 1799, 1798, 1797, 4, 1954, 1235, 1955, 881, 371, 1985, 1986, 1987, 504, 1104, 1682, 748, 749, 1877, 747, 163, 1882, 1883, 1239, 1971, 1972, 353, 985, 1973, 1974, 1240, 1975, 1976, 1241, 1977, 1978, 1979, 899, 982, 1909, 1159, 979, 1910, 1227, 1911, 1912, 1228, 1913, 1914, 852, 490, 1915, 1916, 49, 1933, 1934, 5, 1648, 1649, 459, 1650, 1160, 1855, 1035, 1312, 388, 1939, 1940, 863, 913, 1005, 1226, 1908, 1225, 1907, 1906, 173, 1905, 1186, 1734, 1185, 464, 1861, 1860, 1232, 1936, 1937, 1938, 1101, 1474, 1039, 927, 1753, 1754, 478, 1881, 1880, 1879, 1878, 752, 751, 1037, 1317, 127, 1444, 1445, 1089, 452, 1872, 497, 46, 1832, 1209, 1831, 1161, 1654, 460, 1809, 1133

In [ ]:
def has_hamiltonian_path_via_cycle(G, A, cycle_nodes, B):
    return any(G.has_edge(A, node) for node in cycle_nodes) and any(G.has_edge(B, node) for node in cycle_nodes)

def contract_cycle_via_posa(G, cycles):
    mapping = {}
    valid_cycles = {}
    for i, cycle in enumerate(cycles):
        s_node = f"supernode_{i}"
        for node in cycle:
            mapping[node] = s_node
        valid_cycles[s_node] = cycle
    H = nx.Graph()
    for u, v, data in G.edges(data=True):
        new_u = mapping.get(u, u)
        new_v = mapping.get(v, v)
        if new_u != new_v:
            H.add_edge(new_u, new_v, **data)
    to_contract = []
    for node in list(H.nodes()):
        if isinstance(node, str) and node.startswith("supernode_"):
            nbrs = list(H.neighbors(node))
            if len(nbrs) == 2:
                A, B = nbrs
                if has_hamiltonian_path_via_cycle(G, A, valid_cycles[node], B):
                    to_contract.append((node, A, B))
    for s_node, A, B in to_contract:
        if H.has_node(s_node):
            H.remove_node(s_node)
            H.add_edge(A, B)
    return H

def chain_contraction_preserving(G):
    H = G.copy()
    changed = True
    while changed:
        changed = False
        for node in list(H.nodes()):
            if isinstance(node, str) and node.startswith("supernode_"):
                continue
            if H.degree(node) == 2:
                nbrs = list(H.neighbors(node))
                if len(nbrs) == 2:
                    A, B = nbrs
                    H.remove_node(node)
                    H.add_edge(A, B)
                    changed = True
    return H

def perturbation_reduction(G):
    n_threshold = np.log(G.number_of_nodes()) * 50
    contracted = G.copy()
    new_contracted = nx.Graph()
    while abs(contracted.number_of_nodes() - new_contracted.number_of_nodes()) < n_threshold:
        cycles = pathfinders.greedy_partial_cycle_extractor(contracted, {})  # You may pass fiedler info if available.
        contracted = contract_cycle_via_posa(contracted, cycles) if cycles else contracted.copy()
        new_contracted = chain_contraction_preserving(contracted)
    return new_contracted

if __name__ == '__main__':
    G = nx.erdos_renyi_graph(100, 0.05)
    # In a symmetric graph the Fiedler vector is generally centered; if desired,
    # you can take absolute values for ranking. Adjust based on your needs.
    G_reduced = perturbation_reduction(G)
    print("Nodes:", list(G_reduced.nodes()))
    print("Edges:", list(G_reduced.edges()))
        

Graph with 2000 nodes and 3996 edges
iteration 1
extracted cycle: [1792, 1791, 1790, 1789, 1203, 1956, 1957, 1236, 15, 1980, 1981, 1242, 1982, 1243, 1983, 1244, 1984, 503, 838, 1989, 1245, 1990, 1991, 1992, 1246, 1993, 1994, 1995, 1171, 1691, 1692, 1693, 1173, 1917, 1918, 499, 1919, 500, 1920, 1921, 1229, 1922, 501, 1988, 1184, 1725, 773, 774, 1941, 998, 1202, 1799, 1798, 1797, 4, 1954, 1235, 1955, 881, 371, 1985, 1986, 1987, 504, 1104, 1682, 748, 749, 1877, 747, 163, 1882, 1883, 1239, 1971, 1972, 353, 985, 1973, 1974, 1240, 1975, 1976, 1241, 1977, 1978, 1979, 899, 982, 1909, 1159, 979, 1910, 1227, 1911, 1912, 1228, 1913, 1914, 852, 490, 1915, 1916, 49, 1933, 1934, 5, 1648, 1649, 459, 1650, 1160, 1855, 1035, 1312, 388, 1939, 1940, 863, 913, 1005, 1226, 1908, 1225, 1907, 1906, 173, 1905, 1186, 1734, 1185, 464, 1861, 1860, 1232, 1936, 1937, 1938, 1101, 1474, 1039, 927, 1753, 1754, 478, 1881, 1880, 1879, 1878, 752, 751, 1037, 1317, 127, 1444, 1445, 1089, 452, 1872, 497, 46, 1832, 1209, 18

In [18]:
results = []
for name, graph in nonham_tests:
    print(graph)
    results.append(perturbation_reduction(graph))

Graph named 'Petersen Graph' with 10 nodes and 15 edges


TypeError: 'int' object is not iterable

In [9]:
import heapq

def extract_frontier(cycle_nodes, all_nodes_set, ncache):
    return {(u, v) for u in cycle_nodes for v in (all_nodes_set - ncache[u]) }

def compute_frontier_quality(G, cycle_nodes, all_nodes_set, ncache, deg_cache):
    frontier = extract_frontier(cycle_nodes, all_nodes_set, ncache)
    if not frontier:
        return 0.0, frontier
    frontier_fraction = len(frontier) / len(cycle_nodes)
    total_deg = sum(deg_cache.get(u, 0) for u, _ in frontier)
    avg_frontier_deg = total_deg / len(frontier)
    quality = frontier_fraction * avg_frontier_deg
    return quality, frontier

def greedy_fiedler_cycle_extractor_stop_at_threshold(G, fiedler, min_cycle_size=40, lower_cycle_bound=20, min_frontier_quality_factor=0.4):
    cycles, frontiers_list, stray_paths = [], [], []
    visited_global = set()
    nodes = list(G.nodes())
    all_nodes_set = set(nodes)
    ncache = {node: set(G.neighbors(node)) for node in nodes}
    deg_cache = {node: G.degree(node) for node in nodes}
    fiedler_map = dict(zip(nodes, fiedler))
    avg_deg_G = sum(deg_cache.values()) / len(deg_cache)
    quality_threshold = min_frontier_quality_factor * avg_deg_G
    for start in nodes:
        if start in visited_global:
            continue
        path = [start]
        visited_local = {start}
        current = start
        candidate_cycle = None
        candidate_frontier = None
        while True:
            best_diff, next_node = float('inf'), None
            for nbr in G.neighbors(current):
                if nbr not in visited_local:
                    diff = abs(fiedler_map.get(nbr, 0) - fiedler_map.get(current, 0))
                    if diff < best_diff:
                        best_diff, next_node = diff, nbr
            if next_node is None:
                break
            path.append(next_node)
            visited_local.add(next_node)
            current = next_node
            if len(path) >= lower_cycle_bound and G.has_edge(current, start):
                quality, frontier = compute_frontier_quality(G, path, all_nodes_set, ncache, deg_cache)
                if quality >= quality_threshold:
                    candidate_cycle = path + [start]
                    candidate_frontier = frontier
                    break
            if len(path) >= min_cycle_size and G.has_edge(current, start):
                candidate_cycle = path + [start]
                candidate_frontier = frontier
                break
        if candidate_cycle is not None:
            cycles.append(candidate_cycle)
            frontiers_list.append(candidate_frontier)
            visited_global.update(path)
        else:
            stray_paths.append(path)
            visited_global.update(path)
    stray_vertices = all_nodes_set - visited_global
    residual = {"paths": stray_paths, "vertices": stray_vertices}
    return cycles, frontiers_list, residual

def augment_frontiers(G, frontiers_list, residual):
    residual_frontiers = []
    print(len(residual.items()))
    for path in residual.get("paths", []):
        path_set = set(path)
        res_f = set()
        for u in path:
            for nbr in G.neighbors(u):
                if nbr not in path_set:
                    res_f.add((u, nbr))
        print(f"residual path {path} frontier: {res_f}")
        residual_frontiers.append(res_f)
    stray_frontiers = []
    for node in residual.get("vertices", set()):
        res_f = {(node, nbr) for nbr in G.neighbors(node)}
        print(f"residual vertex {node} frontier: {res_f}")
        stray_frontiers.append(res_f)
    aug = frontiers_list.copy()
    aug.extend(residual_frontiers)
    aug.extend(stray_frontiers)
    return aug

def create_whole_mapping(cycles, residual):
    mapping = {}
    for i, cycle in enumerate(cycles):
        label = f"c{i}"
        nodes = set(cycle[:-1]) if cycle[0] == cycle[-1] else set(cycle)
        for node in nodes:
            mapping[node] = label
    for i, path in enumerate(residual.get("paths", [])):
        label = f"r{i}"
        for node in path:
            mapping[node] = label
    for node in residual.get("vertices", set()):
        mapping[node] = f"s_{node}"
    return mapping

def neighbor_cache_from_graph(G):
    return {node: set(G.neighbors(node)) for node in G.nodes()}

def recompute_frontier_cached(nodes, G, ncache):
    f = set()
    for u in nodes:
        for v in ncache[u]:
            if v not in nodes:
                f.add((u, v))
    return f

def create_supernodes(cycles, residual, G):
    mapping = create_whole_mapping(cycles, residual)
    clusters = {}
    for node, label in mapping.items():
        clusters.setdefault(label, set()).add(node)
    ncache = neighbor_cache_from_graph(G)
    supernodes = {}
    for label, nodes in clusters.items():
        f = recompute_frontier_cached(nodes, G, ncache)
        supernodes[label] = {"label": label, "nodes": nodes, "frontier": f, "cycle": None}
    return list(supernodes.values()), mapping

def connections_from_frontiers(frontiers, mapping):
    connd = {}
    for frontier in frontiers:
        for u, v in frontier:
            src = mapping.get(u)
            tgt = mapping.get(v)
            if src is None or tgt is None or src == tgt:
                continue
            key = tuple(sorted((src, tgt)))
            connd.setdefault(key, []).append((u, v))
    return connd

def candidate_edges_between(snA, snB):
    return [edge for edge in snA["frontier"] if edge[1] in snB["nodes"]]

def merge_supernodes(snA, snB, G, ncache):
    merged_nodes = snA["nodes"] | snB["nodes"]
    merged_cycle = snA["cycle"] + snB["cycle"][1:] if (snA.get("cycle") and snB.get("cycle")) else None
    merged_frontier = recompute_frontier_cached(merged_nodes, G, ncache)
    return {"label": None, "nodes": merged_nodes, "cycle": merged_cycle, "frontier": merged_frontier}

def rotate_cycle_to(cycle, x):
    idx = cycle.index(x)
    new_cycle = cycle[idx:] + cycle[1:idx+1]
    if new_cycle[0] != new_cycle[-1]:
        new_cycle.append(new_cycle[0])
    return new_cycle

def sew_two_cycles(cycleA, cycleB, candidates):

    if len(candidates) < 2:
        raise ValueError("must provide at least two candidate edge pairs.")
    
    a1, b1 = candidates[0]
    a2, b2 = candidates[1]

    A_rot = rotate_cycle_to(cycleA, a1)
    B_rot = rotate_cycle_to(cycleB, b1)
    
    try:
        idx_a2 = A_rot.index(a2)
    except ValueError:
        raise ValueError(f"Vertex {a2} not found in rotated cycle A.")
    
    try:
        idx_b2 = B_rot.index(b2)
    except ValueError:
        raise ValueError(f"Vertex {b2} not found in rotated cycle B.")
    new_cycle = A_rot[:idx_a2+1] + list(reversed(B_rot[:idx_b2+1]))
    
    if new_cycle[0] != new_cycle[-1]:
        new_cycle.append(new_cycle[0])
    return new_cycle

def try_sew_cycle(cycleA, cycleB, cand_list):
    for i in range(len(cand_list)):
        for j in range(i+1, len(cand_list)):
            try:
                sewn_cycle = sew_two_cycles(cycleA, cycleB, [cand_list[i], cand_list[j]])
                return sewn_cycle
            except Exception:
                continue
    return None

def iterative_sewing(supernodes, connection_dict, G, mapping):
    sn_map = {sn["label"]: sn for sn in supernodes}
    ncache = neighbor_cache_from_graph(G)
    heap = []
    for key, cand in connection_dict.items():
        if len(cand) >= 2:
            heapq.heappush(heap, (len(cand), key, cand))
    while heap:
        cost, (l1, l2), cand_list = heapq.heappop(heap)
        if l1 not in sn_map or l2 not in sn_map:
            continue
        cycleA = sn_map[l1]["cycle"]
        cycleB = sn_map[l2]["cycle"]
        new_cycle = try_sew_cycle(cycleA, cycleB, cand_list)
        if new_cycle is None:
            continue
        new_label = f"{l1}+{l2}"
        # Instead of using only new_cycle's nodes, store the union of the two merged supernodes
        new_nodes = sn_map[l1]["nodes"] | sn_map[l2]["nodes"]
        new_supernode = {
            "label": new_label,
            "cycle": new_cycle,
            "nodes": new_nodes,
            "frontier": recompute_frontier_cached(new_nodes, G, ncache)
        }
        for node in new_supernode["nodes"]:
            mapping[node] = new_label
        del sn_map[l1]
        del sn_map[l2]
        sn_map[new_label] = new_supernode
        keys_to_remove = [k for k in connection_dict if l1 in k or l2 in k]
        for k in keys_to_remove:
            del connection_dict[k]
        for other in sn_map:
            if other == new_label:
                continue
            new_cand = candidate_edges_between(new_supernode, sn_map[other])
            if new_cand and len(new_cand) >= 2:
                key = tuple(sorted((new_label, other)))
                connection_dict[key] = new_cand
        heap = []
        for key, cand in connection_dict.items():
            if len(cand) >= 2:
                heapq.heappush(heap, (len(cand), key, cand))
    return list(sn_map.values())

In [10]:
def full_core_sewing(G, fiedler, min_cycle_size=40, lower_cycle_bound=20, min_frontier_quality_factor=0.4):
    cycles, frontiers, residual = greedy_fiedler_cycle_extractor_stop_at_threshold(G, fiedler, min_cycle_size, min_frontier_quality_factor)
    # we only use the cycles for core sewing – we ignore residuals -- may be interesting to integrate residuals for certain density thresholds ?
    mapping = create_whole_mapping(cycles, {"paths": [], "vertices": set()})
    supernodes, mapping = create_supernodes(cycles, {"paths": [], "vertices": set()}, G)
    connection_dict = connections_from_frontiers(frontiers, mapping)
    sewn_supernodes = iterative_sewing(supernodes, connection_dict, G, mapping)
    if sewn_supernodes:
        final_supernode = max(sewn_supernodes, key=lambda sn: len(sn["nodes"]))
    else:
        final_supernode = None
    return final_supernode, residual

In [11]:
def test_suspectham(G):
    A = se.to_adjacency(G)
    _,fiedler,_ = se.power_method(A)
    return full_core_sewing(G, fiedler)

In [ ]:
G = random.choice(ham_tests)
final_core, residual = test_suspectham(G)

In [ ]:
print(residual)

{'paths': [], 'vertices': set()}
